# The LangGraph Framework

## Introduction to LangGraph

`LangGraph` is a framework that allows us to build production-ready applications by giving us control tools over the flow of our agents. `LangGraph` is a framework developed by `LangChain` to **manage the control flow of applications that integrate an LLM.**
- LangChain provides a standard interface to interact with models and other components, useful for retrieval, LLM calls, and tool calls.

`CodeAgent` in `smolagents` are very free. They can call multiple tools in a single action step, create their own tools, etc. However, this behavior can make them less predictable and less controllable than a regular Agent workfing with JSON.

`LangGraph` is preferred if we need "control" on the execution of our agent.

Key benefits from `LangGraph`:
- Multi-step reasoning processes that need explicit control on the flow
- Applications requiring persistence of state between steps
- Systems that combine deterministic logic with AI capabilities
- Workflows that need human-in-the-loop interventions
- Complex agent architectures with multiple components working together


At its core, `LangGraph` uses a directed graph structure to deine the flow of our application
- **Nodes** represent individiual processing step (like calling an LLM, using a tool, or making a decision)
- **Edges** define the possible transitions between steps
- **State** is user-defined and maintained and passed between nodes during execution. When deciding which node to target next, this is the current state that we look at.


## Building Blocks of LangGraph

### State

**State** is the central concept in LangGraph. It represents all the information that flows through our application.

In [ ]:
from typing_extensions import TypedDict

class State(TypedDict):
    graph_state: str

The state is **user-defined**, hence the fields should be carefully crafted to contain all data needed for decision-making process.


We need to think carefully about what information our application needs to track between steps.

### Nodes

**Nodes** are Python functions. Each node
- takes the state as input
- performs some operation
- returns updates to the state

In [ ]:
def node_1(state):
    print('---Node 1---')
    return {'graph_state': state['graph_state'] + " I am"}

def node_2(state):
    print('---Node 2---')
    return {'graph_state': state['graph_state'] + " happy!"}

def node_3(state):
    print('---Node 3---')
    return {'graph_state': state['graph_state'] + " sad!"}

Nodes can contain
- LLM calls: generate text or make decisions
- tool calls: interact with external systems
- conditional logic: determine next steps
- human intervention: get input from users

### Edges

**Edges** connect nodes and define the possible paths through our graph.

In [ ]:
import random
from typing import Literal


def decide_mood(state) -> Literal["node_2", "node_3"]:
    # We will use state to decide on the next node to visit
    user_input = state['graph_state']

    # here, we do a 50/50 split between node 2 and node 3
    if random.random() < 0.5:
        return "node_2"
    else:
        return "node_3"

Edges can be
- direct: always go from node A to node B
- conditional: choose the next node based on the current state

### StateGraph

The **StateGraph** is the container that holds our entire agent workflow.

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END


# Build graph
builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Logic
builder.add_edge(START, "node_1")
builder.add_conditional_edges('node_1', decide_mood)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)

# Build graph
graph = builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

We can use the graph now:

In [ ]:
graph.invoke(
    {'graph_state': "Hi, this is Bin."}
)

## Building Our First LangGraph

We will implement an email processing agent to
- read incoming emails
- classify them as spam or legitimate
- draft a prelimiary response for legitimate emails
- send information to a specific receipent when legitimate

In [ ]:
!pip install -qU langgraph langchain_openai langchain_huggingface

In [ ]:
import os
from typing import TypedDict, List, Dict, Any, Optional
import langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage

from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

### Define the State

First, we will define what information the agent needs to track during the email processing workflow:

In [ ]:
class EmailState(TypedDict):
    # The email being processed
    email: Dict[str, Any] # contains subject, sender, body, etc.

    # Category of the email (inquery, complaint, etc.)
    email_category: Optional[str]

    # Reason why the email was marked as spam
    spam_reason: Optional[str]

    # Analysis and decisions
    is_pam: Optional[bool]

    # Response generation
    draft_response: Optional[str]

    # Processing metadata
    messages: List[Dict[str, Any]] # track conversation with LLM for analysis

We want to make our state comphrehensive enough to track all the important information, but avoid bloating it with unnecessary details.

### Define the Nodes

In [ ]:
# Initialize the LLM
model = ChatOpenAI(model='gpt-4o', temperature=0)


def read_email(state: EmailState):
    """Read and log the incoming emails"""
    email = state['email']

    # we might do some initial processing
    print(f"Agent is processing an email from {email['sender']} with subject: {email['subject']}")

    # No state changes needed here
    return {}


def classify_email(state: EmailState):
    """Use an LLM to determine if the email is spam or legitimate"""
    email = state['email']

    # Prepare prompt for the LLM
    prompt = f"""
    As AI assistant, analyze this email and determine if it is spam or legitimate.

    Email:
    From: {email['sender']}
    Subject: {email['subject']}
    Body: {email['body']}

    First, determine if this email is spam. If it is spam, explain why.
    If it is legitimate, categorize it (inquiry, complaint, thank you, etc.)."""

    # Call the LLM
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)

    # Simple logic to parse the response
    response_text = response.content.lower()
    is_spam = 'spam' in response_text and 'ham' not in response_text

    # Extract a reason if it is spam
    spam_reason = None
    if is_spam and "reason:" in response_text:
        spam_reason = response_text.split("reason:")[1].strip()

    # Determine category if legitimate
    email_category = None
    if not is_spam:
        categories = ['inquiry', 'complaint', 'thank you', 'request', 'information']
        for category in categories:
            if category in response_text:
                email_category = category
                break

    # Update messages for tracking
    new_messages = state.get('messages', []) + [
        {'role': 'user', 'content': prompt},
        {'role': 'assistant', 'content': response.content}
    ]

    # Return state updates
    return {
        'is_spam': is_spam,
        'spam_reason': spam_reason,
        'email_category': email_category,
        'messages': new_messages
    }


def handle_spam(state: EmailState):
    """Discard spam email with a note"""
    print(f"Agent has marked the email as spam. Reason: {state['spam_reason']}")
    print("The email has been moved to the spam folder.")

    return {}


def draft_response(state: EmailState):
    """Draft a preliminary response for legitimate emails"""
    email = state['email']
    category = state['email_category'] or "general"

    # Prepare prompt for the LLM
    prompt = f"""
    As AI assistant, draft a polite preliminary response to this email.

    Email:
    From: {email['sender']}
    Subject: {email['subject']}
    Body: {email['body']}

    This email has been categorized as: {category}

    Draft a brief, professional response that Mr. Hugg can review and personalize before sending."""

    # Call the LLM
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)

    # Update messages for tracking
    new_messages = state.get('messages', []) + [
        {'role': 'user', 'content': prompt},
        {'role': 'assistant', 'content': response.content}
    ]

    # Return state updates
    return {
        'draft_response': response.content,
        'messages': new_messages
    }


def notify_mr_wayne(state: EmailState):
    """Notify Mr. Wayne about the email and presents the draft response"""
    email = state['email']

    print("\n" + "="*50)
    print(f"Sir, you've received an email from {email['sender']}.")
    print(f"Subject: {email['subject']}")
    print(f"Category: {state['email_category']}")
    print("\nI've prepared a draft response for your review:")
    print("-"*50)
    print(state["email_draft"])
    print("="*50 + "\n")

    return {}

### Define Routing Logic

We need a function to determine which path to take after classification.

In [ ]:
def route_email(state: EmailState) -> str:
    """Determine the next step based on spam classification"""
    if state['is_spam']:
        return "spam"
    else:
        return 'legitimate'

This routing function is called by LangGraph to determine which edge to follow after the classification node. The return value must match one of the keys in our conditional edges mapping.

### Create the StateGraph and Define Edges

In [ ]:
# Create the graph
email_graph = StateGraph(EmailState)

# Add nodes
email_graph.add_node('read_email', read_email)
email_graph.add_node('classify_email', classify_email)
email_graph.add_node('handle_spam', handle_spam)
email_graph.add_node('draft_response', draft_response)
email_graph.add_node('notify_mr_wayne', notify_mr_wayne)

# Start the edge
email_graph.add_edge(START, 'read_email')
# Add edges - defining the flow
email_graph.add_edge('read_email', 'classify_email')
# Add conditional branching from `classify_email`
email_graph.add_conditional_edges(
    'classify_email',
    route_email,
    {
        'spam': 'handle_spam',
        'legitimate': 'draft_response'
    }
)

# Add the final edges
email_graph.add_edge('handle_spam', END)
email_graph.add_edge('draft_response', 'notify_mr_wayne')
email_graph.add_edge('notify_mr_wayne', END)

# Compile the graph
email_graph = email_graph.compile()

In [ ]:
from IPython.display import Image, display

display(Image(compiled_graph.get_graph().draw_mermaid_png()))

### Run the App

In [ ]:
# Example legitimate email
legitimate_email = {
    "sender": "john.smith@example.com",
    "subject": "Question about your services",
    "body": "Dear Mr. Hugg, I was referred to you by a colleague and I'm interested in learning more about your consulting services. Could we schedule a call next week? Best regards, John Smith"
}

# Example spam email
spam_email = {
    "sender": "winner@lottery-intl.com",
    "subject": "YOU HAVE WON $5,000,000!!!",
    "body": "CONGRATULATIONS! You have been selected as the winner of our international lottery! To claim your $5,000,000 prize, please send us your bank details and a processing fee of $100."
}

# Process the legitimate email
print("\nProcessing legitimate email...")
legitimate_result = compiled_graph.invoke({
    "email": legitimate_email,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": []
})

# Process the spam email
print("\nProcessing spam email...")
spam_result = compiled_graph.invoke({
    "email": spam_email,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": []
})

### Inspect Agent with Langfuse

In [ ]:
!pip install -qU langfuse langchain

In [ ]:
import os
from google.colab import userdata

os.environ["LANGFUSE_PUBLIC_KEY"] = userdata.get('LANGFUSE_PUBLIC_KEY')
os.environ["LANGFUSE_SECRET_KEY"] = userdata.get('LANGFUSE_SECRET_KEY')
os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

In [ ]:
from langfuse.callback import CallbackHandler

# Initialize Langfuse CallbackHandler for LangGraph/Langchain (tracing)
langfuse_handler = CallbackHandler()

# Process legitimate email
legitimate_result = compiled_graph.invoke(
    input={"email": legitimate_email, "is_spam": None, "spam_reason": None, "email_category": None, "draft_response": None, "messages": []},
    config={"callbacks": [langfuse_handler]}
)

## Document Analysis Graph

We will implement a document analysis agent using LangGraph. The agent can
- process images document
- extract text using vision models
- perform calculations when needed
- analyze content and provide concise summaries
- execute specific instructions related to documents

In [ ]:
!pip install -qU langchain_openai langchain_core langgraph

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
import base64
from typing import List, TypedDict, Annotated, Optional
from langchain_openai import ChatOpenAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage
from langgraph.graph.message import add_messages
from langgraph.graph improt START, END, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display

### Define Agent's State

`AnyMessage` is a class from LangChain taht defines messages, and `add_messages` is an operator that adds the latest message rather than overwriting it with the latest state.

In the agent's state, we can add operators in our state to define the way they should interact together.

In [ ]:
class AgentState(TypedDict):
    # The document provided
    input_file: Optional[str] # contains file path (PDF/PNG)
    messages: Annotated[list[AnyMessage], add_messages]

### Prepare Tools

In [ ]:
vision_llm = ChatOpenAI(model='gpt-4o')


def extract_text(img_path: str) -> str:
    """Extract text from an image file using a multimodal model."""
    all_text = ""
    try:
        # Read image and encode as base64
        with open(img_path, 'rb') as image_file:
            image_bytes = image_file.read()

        image_base64 = base64.b64encode(image_bytes).decode('utf-8')

        # Prepare the prompt including the base64 image data
        message = [
            HumanMessage(
                content=[
                    {
                        'type': 'text',
                        'text': "Extract all the text from this image. Return only the extracted text, no explanations."
                    },
                    {
                        'type': 'image_url':
                        'image_url': {'url': f'data:image/png;base64,{image_base64}'}
                    }
                ]
            )
        ]

        # Call the vision-capable model
        response = vision_llm.invoke(message)

        # Append extracted text
        all_text += response.content + '\n\n'

        return all_text.strip()
    except Exception as e:
        error_msg = f"Error extracting text: {str(e)}"
        print(error_msg)
        return ""

In [ ]:
llm = ChatOpenAI(model='gpt-4o')

def divide(a: int, b: int) -> float:
    """Divide a and b"""
    return a / b

In [ ]:
# Equip tools
tools = [
    divide,
    extract_text
]

llm_with_tools = llm.bind_tools(
    tools,
    parallel_tool_calls=False
)

### The Nodes

In [ ]:
def assistant(state: AgentState):
    # System message
    textual_description_of_tool = """
    extract_text(img_path: str) -> str:
        Extract text from an image file using a multimodal model.

        Args:
            img_path: A local image file path (strings).

        Returns:
            A single string containing the concatenated text extracted from each image.
    divide(a: int, b: int) -> float:
        Divide a and b"""

    image = state['input_file']
    sys_msg = SystemMessage(
        content=f"""You are a helpful butler named Alfred that serves Mr. Wayne and Batman.
        You can analyse documents and run computations with provided tools:\n{textual_description_of_tool} \n
        You have access to some optional images. Currently the loaded image is: {image}"""
    )

    response = llm_with_tools.invoke([sys_msg] + state['messages'])

    return {
        'messages': [response],
        'input_file': state['input_file']
    }

### the ReAct Pattern

The agent follows the ReAct pattern (Reason-Act-Observe)
- **Reason** about the documents and requests
- **Act** by using appropriate tools
- **Observe** the results
- **Repeat** as necessary until the task is completed

In [ ]:
# The graph
builder = StateGraph(AgentState)

# Define nodes
builder.add_node('assistant', assistant)
builder.add_node('tools', ToolNode(tools)) # tool node

# Define edges
builder.add_edge(START, 'assistant')
builder.add_conditional_edges(
    'assistant',
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition
)

builder.add_edge('tools', 'assistant')
react_graph = builder.compile()

display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

We define a `tools` node with our list of tools. The `assistant` node is just our model with bound tools. We create a graph with `assistant` and `tools` nodes.

We also add `tools_condition` edge, which routes to `END` or to `tools` based on whether the `assistant` calls a tool.

### Test

In [ ]:
messages = [HumanMessage(content="Divide 6790 by 5")]
messages = react_graph.invoke({"messages": messages, "input_file": None})

# Show the messages
for m in messages['messages']:
    m.pretty_print()

In [ ]:
messages = [HumanMessage(content="According to the note provided by Mr. Wayne in the provided images. What's the list of items I should buy for the dinner menu?")]
messages = react_graph.invoke({"messages": messages, "input_file": "Batman_training_and_meals.png"})